# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import re
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_recall_fscore_support,accuracy_score,label_ranking_average_precision_score
import pickle
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mhali\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mhali\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('Disasters', con=engine)
X = df['message']
y = df[df.columns[5:]]
added = pd.get_dummies(df[['related','genre']])
y = pd.concat([y, added], axis=1)

### 2. Write a tokenization function to process your text data

the function below takes in a text and uses the url_regx to tokenize it with the help of tokenizer

In [3]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    # take out all punctuation while tokenizing
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    
    # lemmatize as shown in the lesson
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

random forests classifier is built using pipeline tobe used further

In [4]:
# Create pipeline with Classifier
moc = MultiOutputClassifier(RandomForestClassifier())

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', moc)
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=1361)
pipeline.fit(X_train,y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000242401C4288>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

this function will be called further to get results thiis takes in y_test and y_pred data and prints out f_score,precision and recall

In [13]:
def display_results(Y_test, y_pred):
    result=precision_recall_fscore_support(Y_test, y_pred)
    for i, col in enumerate(Y_test.columns.values):
        accu=accuracy_score(Y_test.loc[:,col],y_pred[:,i])
        score = ('{}\n Accuracy:  {:.4f}   % Precision: {:.4f}   % Recall {:.4f} '.format(
                  col,accu,result[0][i],result[1][i]))
        print(score)
    avg_precision = label_ranking_average_precision_score(Y_test, y_pred)
    avg_score= ('label ranking average precision: {}'.format(avg_precision))
    print(avg_score)

In [9]:
y_pred=pipeline.predict(X_test)

the above fuction is called here to get the results

In [14]:
results = display_results(y_test, y_pred)

results

request
 Accuracy:  0.9034   % Precision: 0.9080   % Recall 0.4535 
offer
 Accuracy:  0.9940   % Precision: 0.0000   % Recall 0.0000 
aid_related
 Accuracy:  0.7756   % Precision: 0.7824   % Recall 0.6467 
medical_help
 Accuracy:  0.9109   % Precision: 0.7083   % Recall 0.0359 
medical_products
 Accuracy:  0.9470   % Precision: 0.6522   % Recall 0.0530 
search_and_rescue
 Accuracy:  0.9727   % Precision: 0.5000   % Recall 0.0915 
security
 Accuracy:  0.9827   % Precision: 0.0000   % Recall 0.0000 
military
 Accuracy:  0.9672   % Precision: 0.6364   % Recall 0.0402 
child_alone
 Accuracy:  1.0000   % Precision: 0.0000   % Recall 0.0000 
water
 Accuracy:  0.9483   % Precision: 0.9000   % Recall 0.2375 
food
 Accuracy:  0.9284   % Precision: 0.8793   % Recall 0.4300 
shelter
 Accuracy:  0.9234   % Precision: 0.8433   % Recall 0.2301 
clothing
 Accuracy:  0.9866   % Precision: 0.7143   % Recall 0.0685 
money
 Accuracy:  0.9793   % Precision: 0.5000   % Recall 0.0093 
missing_people
 Accura

### 6. Improve your model
Use grid search to find better parameters. 

In [15]:
#using grid search
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x00000242401C4288>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x00000242401C4288>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,


the grid search will tune the model for max depth of 10,50 or none and with min sample leaves of 2,5,10

In [16]:
parameters = {'clf__estimator__max_depth': [10, 50, None],
              
              'clf__estimator__min_samples_leaf':[2, 5, 10]}

cv = GridSearchCV(pipeline, parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

the get_results function declared earlier is called upon to get the results

In [19]:
cv.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
results2 = display_results(y_test, y_pred)
results2

C:\Users\mhali\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mhali\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


request
 Accuracy:  0.9034   % Precision: 0.9080   % Recall 0.4535 
offer
 Accuracy:  0.9940   % Precision: 0.0000   % Recall 0.0000 
aid_related
 Accuracy:  0.7756   % Precision: 0.7824   % Recall 0.6467 
medical_help
 Accuracy:  0.9109   % Precision: 0.7083   % Recall 0.0359 
medical_products
 Accuracy:  0.9470   % Precision: 0.6522   % Recall 0.0530 
search_and_rescue
 Accuracy:  0.9727   % Precision: 0.5000   % Recall 0.0915 
security
 Accuracy:  0.9827   % Precision: 0.0000   % Recall 0.0000 
military
 Accuracy:  0.9672   % Precision: 0.6364   % Recall 0.0402 
child_alone
 Accuracy:  1.0000   % Precision: 0.0000   % Recall 0.0000 
water
 Accuracy:  0.9483   % Precision: 0.9000   % Recall 0.2375 
food
 Accuracy:  0.9284   % Precision: 0.8793   % Recall 0.4300 
shelter
 Accuracy:  0.9234   % Precision: 0.8433   % Recall 0.2301 
clothing
 Accuracy:  0.9866   % Precision: 0.7143   % Recall 0.0685 
money
 Accuracy:  0.9793   % Precision: 0.5000   % Recall 0.0093 
missing_people
 Accura

the best estimator obtained from grid search is printed below

In [20]:
#finding the best estimator after using grid search
cv.best_estimator_

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000242401C4288>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(min_samples_leaf=2)))])

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [23]:

moc = MultiOutputClassifier(DecisionTreeClassifier())

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', moc)
    ])

X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train.as_matrix(), y_train.as_matrix())

y_predicted = pipeline.predict(X_test)
results = display_results(y_test, y_predicted)
results

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()


Aggregated f_score is: 0.931533336249
Aggregated precision is: 0.930696028571
Aggregated recall is: 0.932485935385


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if __name__ == '__main__':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.


,Category,f_score,precision,recall
1,request,0.850185,0.849222,0.851236
2,offer,0.990677,0.991121,0.990235
3,aid_related,0.695338,0.694866,0.696063
4,medical_help,0.906385,0.903978,0.909063
5,medical_products,0.942643,0.941661,0.943699
6,search_and_rescue,0.958933,0.957605,0.96033
7,security,0.970095,0.968189,0.972078
8,military,0.965517,0.965517,0.965517
9,child_alone,1,1,1
10,water,0.958696,0.958218,0.959262


### 9. Export your model as a pickle file

In [24]:
pickle.dump(cv, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.